# Popularity Based Recommendation

In [57]:
import pandas as pd
import numpy as np

In [58]:
ratings=pd.read_csv("spark/ml-100k/u.data",header=None)
ratings.head()

,0
0,0\t50\t5\t881250949
1,0\t172\t5\t881250949
2,0\t133\t1\t881250949
3,196\t242\t3\t881250949
4,186\t302\t3\t891717742


In [59]:
ratings=pd.read_csv("spark/ml-100k/u.data",sep="\t")
ratings.head()

,0,50,5,881250949
0,0,172,5,881250949
1,0,133,1,881250949
2,196,242,3,881250949
3,186,302,3,891717742
4,22,377,1,878887116


In [60]:
r_cols=["user_id","movie_id","rating"]
ratings=pd.read_csv("spark/ml-100k/u.data",sep="\t",names=r_cols,usecols=range(3))
print(ratings.head())

m_cols=["movie_id","title"]
movies=pd.read_csv("spark/ml-100k/u.item",sep="|",encoding="iso-8859-1",names=m_cols,usecols=range(2))
print (movies.head())

   user_id  movie_id  rating
0        0        50       5
1        0       172       5
2        0       133       1
3      196       242       3
4      186       302       3
   movie_id              title
0         1   Toy Story (1995)
1         2   GoldenEye (1995)
2         3  Four Rooms (1995)
3         4  Get Shorty (1995)
4         5     Copycat (1995)


In [61]:
ratings=pd.merge(movies,ratings)
ratings.head()

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4
3,1,Toy Story (1995),280,4
4,1,Toy Story (1995),66,3


In [73]:
ratings.title.nunique(),ratings.user_id.nunique()
#number of unique
#1664 film hakkında 944 kişi yorum yapmış

(1664, 944)

In [74]:
movie_grouped=ratings.groupby("title").agg({"rating":[np.size,np.sum,np.mean]})
#agg= filmin ismine göre gruplama yapıyor sonra ratingi ortalaması , toplam ratingi,size=kaç kişi yorum yapmış
#title a göre ratingleri sıralayıp istenenleri alıyoruz

In [75]:
movie_grouped.tail()

rating               
                                        size  sum      mean
title                                                      
Young Guns II (1990)                      44  122  2.772727
Young Poisoner's Handbook, The (1995)     41  137  3.341463
Zeus and Roxanne (1997)                    6   13  2.166667
unknown                                    9   31  3.444444
Á köldum klaka (Cold Fever) (1994)         1    3  3.000000

In [76]:
popular_movies=movie_grouped.sort_values(("rating","mean"),ascending=False)

In [77]:
popular_movies.head()
#Bir kişi 5 oy vermiş ve bir çıkıyor ortalaması 
#bunları kaldırmamız lazım

rating         
                                             size sum mean
title                                                     
They Made Me a Criminal (1939)                  1   5  5.0
Marlene Dietrich: Shadow and Light (1996)       1   5  5.0
Saint of Fort Washington, The (1993)            2  10  5.0
Someone Else's America (1995)                   1   5  5.0
Star Kid (1997)                                 3  15  5.0

In [78]:
grouped_sum=movie_grouped["rating"]["sum"].sum()
grouped_sum
#bütün ratingleri topluyoruz

352997

In [79]:
popular_movies["percentage"]=popular_movies["rating"]["sum"].div(grouped_sum)*100
#burda bütün ratinglerin yüzde kaçını hangi film almış diye düşünüyoruz

In [80]:
popular_movies.head()

rating          percentage
                                             size sum mean           
title                                                                
They Made Me a Criminal (1939)                  1   5  5.0   0.001416
Marlene Dietrich: Shadow and Light (1996)       1   5  5.0   0.001416
Saint of Fort Washington, The (1993)            2  10  5.0   0.002833
Someone Else's America (1995)                   1   5  5.0   0.001416
Star Kid (1997)                                 3  15  5.0   0.004249

In [87]:
popular_movies=popular_movies.sort_values(("percentage"),ascending=False)
#sıralıyoruzs

In [90]:
popular_movies["Rank"]=popular_movies["percentage"].rank(ascending=False, method="min")
#rank verdik

In [91]:
popular_movies.head(20)

rating                 percentage  Rank
                                   size   sum      mean                 
title                                                                   
Star Wars (1977)                    584  2546  4.359589   0.721253   1.0
Fargo (1996)                        508  2111  4.155512   0.598022   2.0
Return of the Jedi (1983)           507  2032  4.007890   0.575642   3.0
Contact (1997)                      509  1936  3.803536   0.548447   4.0
Raiders of the Lost Ark (1981)      420  1786  4.252381   0.505953   5.0
Godfather, The (1972)               413  1769  4.283293   0.501137   6.0
English Patient, The (1996)         481  1759  3.656965   0.498305   7.0
Toy Story (1995)                    452  1753  3.878319   0.496605   8.0
Silence of the Lambs, The (1991)    390  1673  4.289744   0.473942   9.0
Scream (1996)                       478  1645  3.441423   0.466010  10.0
Pulp Fiction (1994)                 394  1600  4.060914   0.453262  11.0
Air Force One (1997)                431  1565  3.631090   0.443347  12.0
Empire Strikes Back, The (1980)     368  1548  4.206522   0.438531  13.0
Liar Liar (1997)                    485  1531  3.156701   0.433715  14.0
Twelve Monkeys (1995)               392  1489  3.798469   0.421817  15.0
Titanic (1997)                      350  1486  4.245714   0.420967  16.0
Independence Day (ID4) (1996)       429  1475  3.438228   0.417851  17.0
Chasing Amy (1997)                  379  1455  3.839050   0.412185  18.0
Jerry Maguire (1996)                384  1425  3.710938   0.403686  19.0
Rock, The (1996)                    378  1396  3.693122   0.395471  20.0

## populer filimlerin listesini çıkardık